# Convert to TFLite

In [ ]:
project_id = datalab_project_id()
bucket = 'gs://%s-image-classifier-lite' % project_id
!gsutil mb $bucket

In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile


def load_graph(graph_filename):
  with tf.gfile.GFile(graph_filename, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  with tf.Graph().as_default() as graph:
    tf.import_graph_def(
      graph_def,
      input_map=None,
      return_elements=None,
      name="prefix",
      op_dict=None,
      producer_op_list=None)
  input_tensor = graph.get_tensor_by_name('prefix/input:0') #input tensor
  output_tensor = graph.get_tensor_by_name('prefix/final_result:0') #output tensor
  return graph, input_tensor, output_tensor


def convert_graph(graph, input_tensor, output_tensor, lite_file):
  with tf.Session(graph=graph) as sess:
    tflite_model = tf.contrib.lite.toco_convert(sess.graph_def,
                                                [input_tensor],
                                                [output_tensor])
    with gfile.GFile(lite_file, "wb") as timeline_file:
      timeline_file.write(tflite_model)


tf.logging.set_verbosity(tf.logging.DEBUG)

model_file = 'gs://candies-ml/model/v2/retrained_graph.pb'
graph, input_tensor, output_tensor = load_graph(model_file)

lite_file = '%s/retrained_graph.lite' % bucket
convert_graph(graph, input_tensor, output_tensor, lite_file)

labels_file = 'gs://candies-ml/model/v2/retrained_labels.txt'
!gsutil cp $labels_file $bucket